# Advent of code - Day12

https://adventofcode.com/2023/day/12

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', True)

import re
from tqdm import tqdm


# Part 1

In [19]:
with open("day12_input.txt") as f:
    lines = f.read().splitlines()

spring_conditions = []
for i, line in enumerate(lines):
    spring_conditions.append(line.split(" "))
    spring_conditions[i][1] = [int(x) for x in spring_conditions[i][1].split(",")]
#spring_conditions
    

In [45]:
def generate_combinations(s):
    if '?' not in s:
        return [s]
    
    i = s.index('?')
    
    with_dot = s[:i] + '.' + s[i+1:]
    with_hash = s[:i] + '#' + s[i+1:]
    
    combinations_with_dot = generate_combinations(with_dot)
    combinations_with_hash = generate_combinations(with_hash)
    
    return combinations_with_dot + combinations_with_hash

def check_combination(combination, criteria):
    if [len(x) for x in re.compile(r'#+').findall(combination)] ==  criteria:
        return True
    else:
        return False

total_combinations = 0
print("Checking total lines", len(spring_conditions))
for i, line in tqdm(enumerate(spring_conditions), desc="Loading..."):
    input_string = line[0]
    criteria = line[1]
    combinations = generate_combinations(input_string)

    count_combinations = 0
    for combination in combinations:
        count_combinations += check_combination(combination, criteria)
    # print(count_combinations)
    total_combinations+=count_combinations

print("Final", total_combinations) 

Checking total lines 1000


Loading...: 1000it [00:24, 40.58it/s]

Final 7716


# Part 2

In [44]:
from functools import cache

@cache
def count_combinations(combination, criteria, value_criteria=0):
    if not combination:
        #print("Not valid combination")
        return not criteria and not value_criteria
    n = 0
    if combination[0] in ("#", "?"): 
        # if first character is a # we should keep checking the first group and the first criteria... recursively
        # print("Expand question mark", combination, criteria, value_criteria, n)
        n += count_combinations(combination[1:], criteria, value_criteria + 1) # increse current counting by one
    if combination[0] in (".", "?") and (criteria and criteria[0] == value_criteria or not value_criteria):
        # only valid combinations are passed, ie, criteria is present, value of the counter is matched or if we did not find #
        #print("Move forward - criteria completed, start new block", combination, criteria, value_criteria, n)
        n += count_combinations(combination[1:], criteria[1:] if value_criteria else criteria)
    return n

with open("day12_input.txt") as f:
    ls = [l.split() for l in f.read().splitlines()]
ls = [(p, tuple(int(n) for n in s.split(","))) for p, s in ls]

print(sum(count_combinations(p + ".", s, 0) for p, s in ls))
print(sum(count_combinations("?".join([p] * 5) + ".", s * 5) for p, s in ls))


7716
18716325559999


Reference: https://www.youtube.com/watch?v=g3Ms5e7Jdqo&ab_channel=HyperNeutrino